In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import AdamW


2024-01-09 12:24:03.117866: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
!nvidia-smi

Tue Jan  9 12:50:38 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   33C    P8    24W / 350W |   1086MiB / 24234MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:

#df = pd.read_excel("wrime3.xlsx",engine = "openpyxl")
#df.head()

In [4]:
# df_train = df[(df["Train/Dev/Test"] == "train") | (df["Train/Dev/Test"] == "dev")]

# train_docs = df_train["Sentence"].tolist()

# train_label1= df_train["Writer_Joy"].tolist()
# train_label2 = df_train["Writer_Sadness"].tolist()
# train_label3 = df_train["Writer_Anticipation"].tolist()
# train_label4 = df_train["Writer_Surprise"].tolist()
# train_label5 = df_train["Writer_Anger"].tolist()
# train_label6 = df_train["Writer_Fear"].tolist()
# train_label7 = df_train["Writer_Disgust"].tolist()
# train_label8 = df_train["Writer_Trust"].tolist()

# #train_labels=[train_label1, train_label2, train_label3, train_label4, train_label5, train_label6,  train_label7, train_label8]

# len(train_docs)
# #print(train_labels)

In [5]:
# train_labels = df_train[['Writer_Joy', 'Writer_Sadness', 'Writer_Anticipation', 'Writer_Surprise', 
#           'Writer_Anger', 'Writer_Fear', 'Writer_Disgust', 'Writer_Trust']].values.tolist()
# #print(train_labels)

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [7]:
model_name = "cl-tohoku/bert-large-japanese"
# model_name = "cl-tohoku/bert-base-japanese-v2"

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=8,problem_type="multi_label_classification")
model = model.to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-large-japanese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

In [8]:
modell = torch.load('multi_pos_weightdayo.pth')


In [9]:
from transformers import TextClassificationPipeline
import torch.nn as nn

class TextClassificationPipelineAddOutputs(TextClassificationPipeline):
    
    def postprocess(self, model_outputs, function_to_apply=None, return_all_scores=False):
        # 既存の処理を行います
        processed = super().postprocess(model_outputs, function_to_apply, return_all_scores)
        
        logits = model_outputs['logits']
        
        logits =torch.sigmoid(logits)
        
        if return_all_scores:
            return [{**proc, **{'logits':logits}} for proc in processed]
        else:
            #print(processed)
            def get_score(data: dict):
                return data['score']
            if len(processed) > 1:
                max_proc = max(processed, key=get_score)
            else:
                max_proc = processed[0]
            return {**max_proc, **{'logits':logits}}
            #return [{**proc, **{'logits':logits}} for proc in processed]

from transformers import TextClassificationPipeline
import torch.nn as nn

class TextClassificationPipelineAddOutputs(TextClassificationPipeline):
    
    def postprocess(self, model_outputs, function_to_apply=None, return_all_scores=False):
        # 既存の処理を行います
        processed = super().postprocess(model_outputs, function_to_apply, return_all_scores)
        
        logits = model_outputs['logits']
        
        logits =torch.sigmoid(logits)
        #print(processed)
        
        if return_all_scores:
            return [{**proc, **{'logits':logits}} for proc in processed]
        else:
            def get_score(data: dict):
                return data['score']
            if len(processed) > 1:
                max_proc = max(processed, key=get_score)
                max2_proc = sorted(processed, key=get_score)[-2] 
                
            else:
                max_proc = processed[0]
                
            #return {**max_proc, **{'logits':logits}}
            #return {**max_proc, **{'logits':logits}},max2_proc
            return {**{max2_proc, max_proc}, **{'logits':logits}}
            
            
            #return [{**proc, **{'logits':logits}} for proc in processed]

In [10]:
from transformers import pipeline
from transformers import TextClassificationPipeline


#sentiment_analyzer = pipeline("sentiment-analysis",model = model.to("cpu"), tokenizer = tokenizer)
sentiment_analyzer =TextClassificationPipelineAddOutputs(model = modell.to("cpu"), tokenizer = tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [11]:
num_labels= {
        "LABEL_0":'喜び',
        "LABEL_1":'悲しみ',
        "LABEL_2":'期待',
        "LABEL_3":'驚き',
        "LABEL_4":'怒り',
        "LABEL_5":'恐れ',
        "LABEL_6":'嫌悪',
        "LABEL_7":'信頼',
}


In [24]:

sentiment_analyzer("驚いたよ")
    

[{'label': 'LABEL_4',
  'score': 0.5619528293609619,
  'logits': tensor([[0.3069, 0.5017, 0.4408, 0.4006, 0.5620, 0.5213, 0.5582, 0.2515]])}]

dict1 = {"a": 1, "b": 2}
dict2 = {"c": 3, "d": 4}
dict3 = {"e": 5, "f": 6}
result = {**dict1, **dict2, **dict3}
print(result)

In [25]:
result = sentiment_analyzer("うん。あなたは安全運転よね。あたしね、さすがに人を乗せたら安全運転だよ。")

print(num_labels[result[0]['label']])


期待


In [12]:
#ketugou = pd.read_excel("ketugou_2_all_henkou.xlsx")
ketugou = pd.read_excel("zentai1.xlsx")
ketugou.head()




,発話者2,応答者2,対話行為,対話行為2
0,どうよ、調子は。,何かねソービジー。,応答(平叙),応答(平叙)
1,そうなん、何で？,大学の先生のお仕事を頼まれたりするんで、それでワープロ打ったりね、今度留学生が小学校で交流会...,応答(平叙),応答(平叙)
2,え、留学生の世話をしろって？,違う。小学校で留学生と交流会をしましょうというやつの、その留学生自体が集まらんもんで、呼んで...,自己開示,応答(YesNo)
3,あー、呼んでこないといけないんだ。,ギャラは出るの？今、食ってるものがギャラなわけ。,質問(YesNo),質問(YesNo)
4,そう、そうなんだ。,あんたのことだでそういうことはないと俺は信じとる。,自己開示,自己開示


In [13]:
ketugou = ketugou.dropna(how="any", axis=0)

In [14]:
ketugou[ketugou.isna().any(axis=1)]

,発話者2,応答者2,対話行為,対話行為2


In [29]:
#ketugou.isnull().sum()
#ketugou.head()


In [16]:
hatuwasya = ketugou["発話者2"].to_list()
kanjyou = []

for j in range(len(hatuwasya)):
    print(j)
    
    result = sentiment_analyzer(hatuwasya[j])
    kanjyou.append(num_labels[result[0]['label']])
    
    
#ketugou["感情"] = kanjyou

#data.to_excel(r + "kanjyou.xlsx", index = False)
#data["感情"].head()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_75/1514962466.py:7 in <cell line: 4>                                       │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_75/1514962466.py'                    │
│ /usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:155  │
│ in __call__                                                                               │
│                                                                                           │
│   152 │   │   │                                                                           │
│   153 │   │   │   If `top_k` is used, one such dictionary is returned per label.          │
│   154 │   │   """                                                                         │
│ ❱ 155 │   │   result = super().__call__(*args, **kwargs)                                  │
│   156 │   │   # TODO try and retrieve it in a nicer way from _sanitize_parameters.        │
│   157 │   │   _legacy = "top_k" not in kwargs                                             │
│   158 │   │   if isinstance(args[0], str) and _legacy:                                    │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1120 in __call__    │
│                                                                                           │
│   1117 │   │   │   │   )                                                                  │
│   1118 │   │   │   )                                                                      │
│   1119 │   │   else:                                                                      │
│ ❱ 1120 │   │   │   return self.run_single(inputs, preprocess_params, forward_params, post │
│   1121 │                                                                                  │
│   1122 │   def run_multi(self, inputs, preprocess_params, forward_params, postprocess_par │
│   1123 │   │   return [self.run_single(item, preprocess_params, forward_params, postproce │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1127 in run_single  │
│                                                                                           │
│   1124 │                                                                                  │
│   1125 │   def run_single(self, inputs, preprocess_params, forward_params, postprocess_pa │
│   1126 │   │   model_inputs = self.preprocess(inputs, **preprocess_params)                │
│ ❱ 1127 │   │   model_outputs = self.forward(model_inputs, **forward_params)               │
│   1128 │   │   outputs = self.postprocess(model_outputs, **postprocess_params)            │
│   1129 │   │   return outputs                                                             │
│   1130                                                                                    │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1026 in forward     │
│                                                                                           │
│   1023 │   │   │   │   inference_context = self.get_inference_context()                   │
│   1024 │   │   │   │   with inference_context():                                          │
│   1025 │   │   │   │   │   model_inputs = self._ensure_tensor_on_device(model_inputs, dev │
│ ❱ 1026 │   │   │   │   │   model_outputs = self._forward(model_inputs, **forward_params)  │
│   1027 │   │   │   │   │   model_outputs = self._e

In [55]:
ketugou["感情"] = kanjyou

In [56]:
ketugou.to_csv("kanjyou_huyo.csv", index = False)

In [39]:
# data = pd.read_excel("data004.txt.xlsx")
# data.head()

In [101]:
# hatuwabun = data['発話文'].to_list()

In [15]:
# for i in range(10):
#     result = sentiment_analyzer(hatuwabun[i])
#     print(num_labels[result[0]['label']],hatuwabun[i])

In [15]:
# kanjyou = []
# for i in range(len(hatuwabun)):
    
#     result = sentiment_analyzer(hatuwabun[i])
#     kanjyou.append(num_labels[result[0]['label']])

In [16]:
# data["感情"] = kanjyou

In [14]:
# data.head()

In [102]:
# import glob

In [13]:
# all_files = glob.glob( "*.txt.xlsx")
# for filename in all_files:
    
#     target = '.'
#     idx = filename.find(target)
#     r = filename[:idx]

#     data = pd.read_excel(filename)

#     hatuwabun = data['発話文'].to_list()

#     kanjyou = []
#     for i in range(len(hatuwabun)):
    
#         result = sentiment_analyzer(hatuwabun[i])
#         kanjyou.append(num_labels[result[0]['label']])
        
#     data["感情"] = kanjyou
    
#     data.to_excel(r + "kanjyou.xlsx", index = False)
#     print(r)
    